# Year 2023 Day 11


In [72]:
import numpy as np
import pandas as pd
import xarray as xr
from pyobsplot import Plot  # , js # can be used to pass js expr as str

from advent_of_code.visualization.observable_plot import visualize_puzzle_input_202311
from advent_of_code.y_2023.problem_202311 import (
    AdventOfCodeProblem202311,
    create_coord_array,
)

### Input Visualization

Note: the following logic can be reused for 202324 (show the vectors aligning progressively)

In [73]:
problem = AdventOfCodeProblem202311()
problem

AdventOfCodeProblem202311(year=2023, day=11)

In [74]:
puzzle_input = problem.parse_input_text_file()
puzzle_input

<xarray.DataArray (row: 140, col: 140)>
array([[46, 46, 46, ..., 46, 46, 46],
       [46, 46, 46, ..., 46, 46, 46],
       [46, 46, 46, ..., 46, 46, 46],
       ...,
       [46, 46, 46, ..., 46, 46, 46],
       [46, 46, 46, ..., 46, 46, 46],
       [46, 46, 46, ..., 46, 35, 46]], dtype=uint8)
Coordinates:
  * row      (row) int64 0 1 2 3 4 5 6 7 8 ... 132 133 134 135 136 137 138 139
  * col      (col) int64 0 1 2 3 4 5 6 7 8 ... 132 133 134 135 136 137 138 139

In [75]:
visualize_puzzle_input_202311(puzzle_input)

ObsplotWidget(spec={'data': [], 'code': {'height': 560, 'width': 560, 'color': {'scheme': 'magma'}, 'x': {'dom…

In [76]:
visualize_puzzle_input_202311(puzzle_input, with_rules=True)

ObsplotWidget(spec={'data': [], 'code': {'height': 560, 'width': 560, 'color': {'scheme': 'magma'}, 'x': {'dom…

Let's count how many nodes there are in this graph:

In [77]:
coord_array = create_coord_array(puzzle_input)
node_count = coord_array["z"].size
node_count

425

Now we can calculate the number of edges in the related [Complete graph](https://en.wikipedia.org/wiki/Complete_graph), where every pair of node is connected:

In [78]:
n = node_count
edge_count = n * (n - 1) // 2  # 2 among n
edge_count

90100

The amount is very high, hence in the visualisation, only a fraction is shown

In [79]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.005,
)

ObsplotWidget(spec={'data': [{'pyobsplot-type': 'DataFrame', 'value': b'ARROW1\x00\x00\xff\xff\xff\xff\x18\x04…

In [80]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.03,
)

ObsplotWidget(spec={'data': [{'pyobsplot-type': 'DataFrame', 'value': b'ARROW1\x00\x00\xff\xff\xff\xff\x18\x04…

We can visually see that there are a lot of distances to compute. The plot is filled with edges. Here is an edge of density of 0.1, we can see that the plot is almost filled with the drawing color:

In [81]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.1,
)

ObsplotWidget(spec={'data': [{'pyobsplot-type': 'DataFrame', 'value': b'ARROW1\x00\x00\xff\xff\xff\xff\x18\x04…

The key to expansion is to assign to every chunk a node (it creates a [Lattice graph](https://en.wikipedia.org/wiki/Lattice_graph)).
Then the same logic can be applied on chunks instead of stars, and the distance
becomes the sum of both adjacency matrix, with the chunks adjacency matrix multiplied by
the expansion coefficient

In [82]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_chunk_graph=True,
)

ObsplotWidget(spec={'data': [], 'code': {'height': 560, 'width': 560, 'color': {'scheme': 'magma'}, 'x': {'dom…

## Problem Solving

In [83]:
from advent_of_code.y_2023.problem_202311 import (
    compute_proximity_matrix_from_coord_array,
    create_chunk_coord_array,
    get_expansion_coef_part_2,
)
import pandas as pd

First, let's visualize again the puzzle input, an boolean array containing the stars.

In [84]:
space_xda = puzzle_input
visualize_puzzle_input_202311(puzzle_input)

ObsplotWidget(spec={'data': [], 'code': {'height': 560, 'width': 560, 'color': {'scheme': 'magma'}, 'x': {'dom…

Then, create an array containing all the coordinates of the stars in the puzzle input

In [85]:
coord_array = create_coord_array(space_xda)
coord_array.to_pandas()

idx,row,col
z,,
0,0,27
1,0,35
2,0,57
3,0,74
4,0,114
...,...,...
420,139,13
421,139,45
422,139,56


Each star is contained in a chunk. Similarly, create an array containing all the _chunks_ coordinates.

In [86]:
chunk_coord_array = create_chunk_coord_array(space_xda, coord_array)
chunk_coord_array.to_pandas()

idx,row,col
z,,
0,0,0
1,0,0
2,0,1
3,0,4
4,0,9
...,...,...
420,8,0
421,8,1
422,8,1


Now, let's compute the _proximity matrix_ for stars, $P_{stars}$


In [87]:
proximity_matrix_stars = compute_proximity_matrix_from_coord_array(coord_array)
proximity_matrix_stars.to_pandas()

compute_proximity_matrix_from_coord_array start
compute_proximity_matrix_from_coord_array end


z,0,1,2,3,4,5,6,7,8,9,...,415,416,417,418,419,420,421,422,423,424
z2,,,,,,,,,,,,,,,,,,,,,
0,0,8,30,47,87,101,24,68,108,21,...,158,146,140,142,172,153,157,168,211,250
1,8,0,22,39,79,93,16,60,100,29,...,166,154,148,142,164,161,149,160,203,242
2,30,22,0,17,57,71,8,38,78,51,...,188,176,170,164,142,183,151,140,181,220
3,47,39,17,0,40,54,25,21,61,68,...,205,193,187,181,151,200,168,157,164,203
4,87,79,57,40,0,14,65,21,21,108,...,245,233,227,221,191,240,208,197,154,163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,153,161,183,200,240,254,175,219,259,142,...,7,7,13,19,49,0,32,43,86,125
421,157,149,151,168,208,222,143,187,227,174,...,39,27,21,15,17,32,0,11,54,93
422,168,160,140,157,197,211,144,176,216,185,...,50,38,32,26,6,43,11,0,43,82


Then, compute the _proximity matrix_ for chunks, $P_{chunks}$

In [88]:
proximity_matrix_chunks = compute_proximity_matrix_from_coord_array(chunk_coord_array)
proximity_matrix_chunks.to_pandas()

compute_proximity_matrix_from_coord_array start
compute_proximity_matrix_from_coord_array end


z,0,1,2,3,4,5,6,7,8,9,...,415,416,417,418,419,420,421,422,423,424
z2,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,4,9,11,1,9,11,0,...,8,8,8,8,9,8,9,9,17,19
1,0,0,1,4,9,11,1,9,11,0,...,8,8,8,8,9,8,9,9,17,19
2,1,1,0,3,8,10,0,8,10,1,...,9,9,9,9,8,9,8,8,16,18
3,4,4,3,0,5,7,3,5,7,4,...,12,12,12,12,11,12,11,11,13,15
4,9,9,8,5,0,2,8,0,2,9,...,17,17,17,17,16,17,16,16,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,8,8,9,12,17,19,9,17,19,8,...,0,0,0,0,1,0,1,1,9,11
421,9,9,8,11,16,18,8,16,18,9,...,1,1,1,1,0,1,0,0,8,10
422,9,9,8,11,16,18,8,16,18,9,...,1,1,1,1,0,1,0,0,8,10


The expansion coefficient $e_{coef}$ is given by the problem description

In [89]:
expansion_coef = get_expansion_coef_part_2()
expansion_coef

999999

The _total proximity matrix_ can then be computed with this expression:
$$
P_{total} = P_{stars} + e_{coef} .  P_{chunks}
$$

In [90]:
total_proximity = proximity_matrix_stars + expansion_coef * proximity_matrix_chunks
total_proximity.to_pandas()

z,0,1,2,3,4,5,6,7,8,9,...,415,416,417,418,419,420,421,422,423,424
z2,,,,,,,,,,,,,,,,,,,,,
0,0,8,1000029,4000043,9000078,11000090,1000023,9000059,11000097,21,...,8000150,8000138,8000132,8000134,9000163,8000145,9000148,9000159,17000194,19000231
1,8,0,1000021,4000035,9000070,11000082,1000015,9000051,11000089,29,...,8000158,8000146,8000140,8000134,9000155,8000153,9000140,9000151,17000186,19000223
2,1000029,1000021,0,3000014,8000049,10000061,8,8000030,10000068,1000050,...,9000179,9000167,9000161,9000155,8000134,9000174,8000143,8000132,16000165,18000202
3,4000043,4000035,3000014,0,5000035,7000047,3000022,5000016,7000054,4000064,...,12000193,12000181,12000175,12000169,11000140,12000188,11000157,11000146,13000151,15000188
4,9000078,9000070,8000049,5000035,0,2000012,8000057,21,2000019,9000099,...,17000228,17000216,17000210,17000204,16000175,17000223,16000192,16000181,8000146,10000153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,8000145,8000153,9000174,12000188,17000223,19000235,9000166,17000202,19000240,8000134,...,7,7,13,19,1000048,0,1000031,1000042,9000077,11000114
421,9000148,9000140,8000143,11000157,16000192,18000204,8000135,16000171,18000209,9000165,...,1000038,1000026,1000020,1000014,17,1000031,0,11,8000046,10000083
422,9000159,9000151,8000132,11000146,16000181,18000193,8000136,16000160,18000198,9000176,...,1000049,1000037,1000031,1000025,6,1000042,11,0,8000035,10000072


Finally, the answer to the problem can be obtained by summing either the lower or upper triangular matrix extracted from $P_{total}$.